In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

In [2]:
# CUDA
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE.type

'cuda'

In [3]:
# Fix random seed
seed = 42
torch.manual_seed(seed)
if DEVICE.type == 'cuda':
	torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Pre-processing input data

In [4]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [5]:
results_df = pd.read_csv('../results/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [6]:
# remove one computer for testing
g_train = results_df[results_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw'])
g_test = results_df[results_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['benchmark','brand_raw'])

In [7]:
mm_df = results_df[results_df['benchmark']=='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
mm_train = mm_df[mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])
mm_test = mm_df[mm_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [8]:
st_df = results_df[results_df['benchmark']!='MATRIX_MULT'].drop(columns=['benchmark'])
# remove one computer for testing
st_train = st_df[st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])
st_test = st_df[st_df['brand_raw'] == '13th Gen Intel(R) Core(TM) i5-1335U'].drop(columns=['brand_raw'])

In [9]:
features = mm_test.columns[1:]
target = mm_test.columns[0]

In [10]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
scaler = StandardScaler()
X_g_train = scaler.fit_transform(X_g_train)
X_g_test = scaler.transform(X_g_test)

## convert to tensor
X_g_train = torch.tensor(X_g_train, dtype=torch.float32).unsqueeze(1)
X_g_test = torch.tensor(X_g_test, dtype=torch.float32).unsqueeze(1)
y_g_train = torch.tensor(y_g_train.values, dtype=torch.float32).view(-1, 1)
y_g_test = torch.tensor(y_g_test.values, dtype=torch.float32).view(-1, 1)

In [11]:
# single thread data
## split data
X_st_train = st_train[features]
y_st_train = st_train[target]

X_st_test = st_test[features]
y_st_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_st_train = scaler.fit_transform(X_st_train)
X_st_test = scaler.transform(X_st_test)

## convert to tensor
X_st_train = torch.tensor(X_st_train, dtype=torch.float32).unsqueeze(1)
X_st_test = torch.tensor(X_st_test, dtype=torch.float32).unsqueeze(1)
y_st_train = torch.tensor(y_st_train.values, dtype=torch.float32).view(-1, 1)
y_st_test = torch.tensor(y_st_test.values, dtype=torch.float32).view(-1, 1)

In [12]:
# multi thread data
## split data
X_mm_train = st_train[features]
y_mm_train = st_train[target]

X_mm_test = st_test[features]
y_mm_test = st_test[target]

## normalize data
scaler = StandardScaler()
X_mm_train = scaler.fit_transform(X_mm_train)
X_mm_test = scaler.transform(X_mm_test)

## convert to tensor
X_mm_train = torch.tensor(X_mm_train, dtype=torch.float32).unsqueeze(1)
X_mm_test = torch.tensor(X_mm_test, dtype=torch.float32).unsqueeze(1)
y_mm_train = torch.tensor(y_mm_train.values, dtype=torch.float32).view(-1, 1)
y_mm_test = torch.tensor(y_mm_test.values, dtype=torch.float32).view(-1, 1)

if DEVICE.type == 'cuda':
	# move to DEVICE
	X_g_train = X_g_train.to(DEVICE)
	y_g_train = y_g_train.to(DEVICE)
	X_g_test = X_g_test.to(DEVICE)
	y_g_test = y_g_test.to(DEVICE)

	X_st_train = X_st_train.to(DEVICE)
	y_st_train = y_st_train.to(DEVICE)
	X_st_test = X_st_test.to(DEVICE)
	y_st_test = y_st_test.to(DEVICE)

	X_mm_train = X_mm_train.to(DEVICE)
	y_mm_train = y_mm_train.to(DEVICE)
	X_mm_test = X_mm_test.to(DEVICE)
	y_mm_test = y_mm_test.to(DEVICE)

# Model

In [13]:
class TransformerModel(nn.Module):
	def __init__(self, input_dim, model_dim, num_heads, num_layers, output_dim, dropout=0.1):
		super(TransformerModel, self).__init__()
		# layers
		self.embedding = nn.Linear(input_dim, model_dim)
		encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, batch_first=True)
		self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
		self.fc = nn.Linear(model_dim, output_dim)
		self.dropout = nn.Dropout(dropout)
	
	def forward(self, x):
		x = self.embedding(x)
		x = self.dropout(x)
		x = self.transformer(x)
		x = self.fc(x.mean(dim=1))
		return x

In [14]:
def objective(trial: optuna.Trial, X_train, y_train, X_test, y_test, input_dim, output_dim):
	# Definimos los hiperparámetros a buscar
	num_heads = trial.suggest_int('num_heads', 1, 8)
	model_dim = trial.suggest_int('model_dim', num_heads * 4, num_heads * 64, step=num_heads)
	num_layers = trial.suggest_int('num_layers', 1, 6)
	dropout = trial.suggest_float('dropout', 0.1, 0.5)
	learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
	weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
	num_epochs = trial.suggest_int('num_epochs', 10, 100)

	# model initialization 
	model = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
	if DEVICE.type == 'cuda':
		model = model.to(DEVICE)
	criterion = nn.MSELoss()
	optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
	# training
	model.train()
	for epoch in range(num_epochs):
		optimizer.zero_grad()
		output = model(X_train)
		loss = criterion(output, y_train)
		loss.backward()
		optimizer.step()
	# evaluation
	model.eval()
	with torch.no_grad():
		predictions = model(X_test)
		val_loss = criterion(predictions, y_test)

		# trial.report(val_loss.item(), epoch+1)
		# if trial.should_prune():
		# 	raise optuna.TrialPruned()
	print(f"Trial: {trial.number} - Loss: {loss.item()} - Val Loss: {val_loss.item()}")
	return val_loss.item()

# Hyperparameters Optimization

In [15]:
n_trials = 100

## General

In [16]:
# configuration optuna
study_g = optuna.create_study(direction='minimize')
study_g.optimize(lambda trial: objective(trial, X_g_train, y_g_train, X_g_test, y_g_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 01:22:53,010] A new study created in memory with name: no-name-b9e37e17-8f00-40f5-b00a-982e98fe202f
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([180, 1])) that is different to the input size (torch.Size([180, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 01:22:53,852] Trial 0 finished with value: 382.2214660644531 and parameters: {'num_heads': 6, 'model_dim':

Trial: 0 - Loss: 116.48963928222656 - Val Loss: 382.2214660644531


[I 2024-06-20 01:22:54,277] Trial 1 finished with value: 346.6871032714844 and parameters: {'num_heads': 1, 'model_dim': 40, 'num_layers': 3, 'dropout': 0.21070024518744457, 'learning_rate': 0.0023484676838076667, 'weight_decay': 0.00012671613027483546, 'num_epochs': 84}. Best is trial 1 with value: 346.6871032714844.


Trial: 1 - Loss: 97.26270294189453 - Val Loss: 346.6871032714844


[I 2024-06-20 01:22:54,521] Trial 2 finished with value: 677.4373779296875 and parameters: {'num_heads': 4, 'model_dim': 236, 'num_layers': 2, 'dropout': 0.2059426149895794, 'learning_rate': 0.00013121736594080206, 'weight_decay': 0.003127410642160959, 'num_epochs': 56}. Best is trial 1 with value: 346.6871032714844.
[I 2024-06-20 01:22:54,710] Trial 3 finished with value: 712.9075317382812 and parameters: {'num_heads': 6, 'model_dim': 264, 'num_layers': 2, 'dropout': 0.16539281254007077, 'learning_rate': 0.00011141973874125393, 'weight_decay': 2.3163982904876242e-05, 'num_epochs': 34}. Best is trial 1 with value: 346.6871032714844.


...

Trial: 96 - Loss: 62.93903732299805 - Val Loss: 96.37464904785156


[I 2024-06-20 01:23:34,807] Trial 97 finished with value: 110.85248565673828 and parameters: {'num_heads': 7, 'model_dim': 434, 'num_layers': 6, 'dropout': 0.2506730895038982, 'learning_rate': 0.0036681148953918684, 'weight_decay': 0.001200152224829185, 'num_epochs': 15}. Best is trial 46 with value: 71.9397201538086.


Trial: 97 - Loss: 47.48685073852539 - Val Loss: 110.85248565673828


[I 2024-06-20 01:23:37,221] Trial 98 finished with value: 140.67703247070312 and parameters: {'num_heads': 8, 'model_dim': 472, 'num_layers': 6, 'dropout': 0.23103645907665055, 'learning_rate': 0.006853117131254777, 'weight_decay': 1.3932977677209717e-05, 'num_epochs': 100}. Best is trial 46 with value: 71.9397201538086.
[I 2024-06-20 01:23:37,364] Trial 99 finished with value: 327.22161865234375 and parameters: {'num_heads': 8, 'model_dim': 224, 'num_layers': 3, 'dropout': 0.27798832468564816, 'learning_rate': 0.004543168074140392, 'weight_decay': 0.00041156840195434233, 'num_epochs': 12}. Best is trial 46 with value: 71.9397201538086.


Trial: 98 - Loss: 46.90190124511719 - Val Loss: 140.67703247070312
Trial: 99 - Loss: 108.8806381225586 - Val Loss: 327.22161865234375


In [17]:
# Results
print(f'Número de pruebas: {len(study_g.trials)}')
trial = study_g.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida: {trial.value}')

Número de pruebas: 100
Mejor prueba: 46
Mejores parametros: {'num_heads': 8, 'model_dim': 224, 'num_layers': 6, 'dropout': 0.2320146403887001, 'learning_rate': 0.009731200416128362, 'weight_decay': 1.8538654992265227e-05, 'num_epochs': 14}
Mejor valor de pérdida: 71.9397201538086


## Single Thread

In [18]:
# configuration optuna
study_st = optuna.create_study(direction='minimize')
study_st.optimize(lambda trial: objective(trial, X_st_train, y_st_train, X_st_test, y_st_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 01:23:37,434] A new study created in memory with name: no-name-27f26d97-d698-4e8b-a710-0582086187aa
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([135, 1])) that is different to the input size (torch.Size([135, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/frnk65/resource-prediction-study/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([15, 1])) that is different to the input size (torch.Size([15, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[I 2024-06-20 01:23:37,819] Trial 0 finished with value: 596.8732299804688 and parameters: {'num_heads': 5, 'model_dim':

Trial: 0 - Loss: 380.5248107910156 - Val Loss: 596.8732299804688
Trial: 1 - Loss: 350.29827880859375 - Val Loss: 555.1475219726562
Trial: 2 - Loss: 365.1371154785156 - Val Loss: 570.3209228515625


[I 2024-06-20 01:23:38,462] Trial 3 finished with value: 33.141597747802734 and parameters: {'num_heads': 7, 'model_dim': 224, 'num_layers': 4, 'dropout': 0.3384521539127864, 'learning_rate': 0.009425342371542283, 'weight_decay': 1.1441429955338632e-05, 'num_epochs': 54}. Best is trial 3 with value: 33.141597747802734.


Trial: 3 - Loss: 17.436294555664062 - Val Loss: 33.141597747802734


[I 2024-06-20 01:23:38,756] Trial 4 finished with value: 131.4488983154297 and parameters: {'num_heads': 4, 'model_dim': 184, 'num_layers': 2, 'dropout': 0.38049056849683727, 'learning_rate': 0.0018259939509228263, 'weight_decay': 6.522779228673137e-05, 'num_epochs': 78}. Best is trial 3 with value: 33.141597747802734.
[I 2024-06-20 01:23:38,866] Trial 5 finished with value: 598.3951416015625 and parameters: {'num_heads': 5, 'model_dim': 50, 'num_layers': 4, 'dropout': 0.1937440343446543, 'learning_rate': 0.00042346111989048353, 'weight_decay': 0.0012221083847293085, 'num_epochs': 16}. Best is trial 3 with value: 33.141597747802734.


...

Trial: 97 - Loss: 28.26228904724121 - Val Loss: 18.258420944213867


[I 2024-06-20 01:24:06,699] Trial 98 finished with value: 20.633153915405273 and parameters: {'num_heads': 7, 'model_dim': 448, 'num_layers': 3, 'dropout': 0.3076866724811859, 'learning_rate': 0.003714759508995781, 'weight_decay': 4.193803122630351e-05, 'num_epochs': 16}. Best is trial 97 with value: 18.258420944213867.


Trial: 98 - Loss: 20.096773147583008 - Val Loss: 20.633153915405273


[I 2024-06-20 01:24:06,914] Trial 99 finished with value: 18.869674682617188 and parameters: {'num_heads': 7, 'model_dim': 448, 'num_layers': 3, 'dropout': 0.2913466650891946, 'learning_rate': 0.00378219057930559, 'weight_decay': 3.93564908797926e-05, 'num_epochs': 17}. Best is trial 97 with value: 18.258420944213867.


Trial: 99 - Loss: 23.403974533081055 - Val Loss: 18.869674682617188


In [19]:
# Results
print(f'Número de pruebas: {len(study_st.trials)}')
trial = study_st.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Número de pruebas: 100
Mejor prueba: 97
Mejores parametros: {'num_heads': 7, 'model_dim': 392, 'num_layers': 3, 'dropout': 0.3360505706259301, 'learning_rate': 0.003928416878757786, 'weight_decay': 3.886347853547162e-05, 'num_epochs': 19}
Mejor valor de pérdida en validación: 18.258420944213867


## Multi Thread

In [20]:
# configuration optuna
study_mm = optuna.create_study(direction='minimize')
study_mm.optimize(lambda trial: objective(trial, X_mm_train, y_mm_train, X_mm_test, y_mm_test, len(features), len(target)), n_trials=n_trials)

[I 2024-06-20 01:24:06,944] A new study created in memory with name: no-name-80584570-e114-4e5a-a0bb-315bf720a6b3
[I 2024-06-20 01:24:07,677] Trial 0 finished with value: 565.8142700195312 and parameters: {'num_heads': 7, 'model_dim': 140, 'num_layers': 4, 'dropout': 0.22565408612120363, 'learning_rate': 1.8041709209664783e-05, 'weight_decay': 0.00036186954849408426, 'num_epochs': 97}. Best is trial 0 with value: 565.8142700195312.


Trial: 0 - Loss: 356.2906494140625 - Val Loss: 565.8142700195312


[I 2024-06-20 01:24:08,142] Trial 1 finished with value: 596.915771484375 and parameters: {'num_heads': 3, 'model_dim': 111, 'num_layers': 3, 'dropout': 0.3008162295685637, 'learning_rate': 1.1775890125225865e-05, 'weight_decay': 0.0017547018318881682, 'num_epochs': 89}. Best is trial 0 with value: 565.8142700195312.


Trial: 1 - Loss: 380.14208984375 - Val Loss: 596.915771484375


[I 2024-06-20 01:24:08,547] Trial 2 finished with value: 548.234375 and parameters: {'num_heads': 2, 'model_dim': 28, 'num_layers': 6, 'dropout': 0.47996037032082217, 'learning_rate': 0.0012146015416587047, 'weight_decay': 0.0009632311858997156, 'num_epochs': 43}. Best is trial 2 with value: 548.234375.


...

Trial: 97 - Loss: 27.91978645324707 - Val Loss: 60.415653228759766


[I 2024-06-20 01:24:51,437] Trial 98 finished with value: 35.96021270751953 and parameters: {'num_heads': 5, 'model_dim': 310, 'num_layers': 6, 'dropout': 0.4253063268852321, 'learning_rate': 0.005612825073207685, 'weight_decay': 0.0007357129787785941, 'num_epochs': 23}. Best is trial 49 with value: 18.234451293945312.


Trial: 98 - Loss: 20.196374893188477 - Val Loss: 35.96021270751953


[I 2024-06-20 01:24:52,132] Trial 99 finished with value: 31.852649688720703 and parameters: {'num_heads': 6, 'model_dim': 330, 'num_layers': 6, 'dropout': 0.45456156631975075, 'learning_rate': 0.004867028636817728, 'weight_decay': 0.0019897507015170113, 'num_epochs': 43}. Best is trial 49 with value: 18.234451293945312.


Trial: 99 - Loss: 17.811973571777344 - Val Loss: 31.852649688720703


In [21]:
# Results
print(f'Trials quantity: {len(study_mm.trials)}')
trial = study_mm.best_trial
print(f'Mejor prueba: {trial.number}')
print(f'Mejores parametros: {trial.params}')
print(f'Mejor valor de pérdida en validación: {trial.value}')

Trials quantity: 100
Mejor prueba: 49
Mejores parametros: {'num_heads': 8, 'model_dim': 264, 'num_layers': 4, 'dropout': 0.35614664010047575, 'learning_rate': 0.009950757747095477, 'weight_decay': 0.0008579245645970467, 'num_epochs': 11}
Mejor valor de pérdida en validación: 18.234451293945312


# Training

In [22]:
input_dim = len(features)
output_dim = 1

## General

In [23]:
# hyperparameters
num_heads = study_g.best_trial.params['num_heads']
model_dim = study_g.best_trial.params['model_dim']
num_layers = study_g.best_trial.params['num_layers']
dropout = study_g.best_trial.params['dropout']
lr = study_g.best_trial.params['learning_rate']
wd = study_g.best_trial.params['weight_decay']
num_epochs = study_g.best_trial.params['num_epochs']

study_g.best_trial.params

{'num_heads': 8,
 'model_dim': 224,
 'num_layers': 6,
 'dropout': 0.2320146403887001,
 'learning_rate': 0.009731200416128362,
 'weight_decay': 1.8538654992265227e-05,
 'num_epochs': 14}

In [24]:
# general model initialization
model_g = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_g = model_g.to(DEVICE)
criterion_g = nn.MSELoss()
optimizer_g = optim.AdamW(model_g.parameters(), lr=lr, weight_decay=wd)

model_g.train()

for epoch in range(num_epochs):
	optimizer_g.zero_grad()
	output = model_g(X_g_train)
	loss = criterion_g(output, y_g_train)
	loss.backward()
	optimizer_g.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_g.eval()
		with torch.no_grad():
			val_predictions = model_g(X_g_test)
			val_loss = criterion_g(val_predictions, y_g_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_g.train()

Epoch 10/14, Loss: 55.85844802856445, Val Loss: 84.19432830810547
Epoch 14/14, Loss: 59.4248046875, Val Loss: 95.29679870605469


## Single Thread

In [25]:
# hyperparameters
num_heads = study_st.best_trial.params['num_heads']
model_dim = study_st.best_trial.params['model_dim']
num_layers = study_st.best_trial.params['num_layers']
dropout = study_st.best_trial.params['dropout']
lr = study_st.best_trial.params['learning_rate']
wd = study_st.best_trial.params['weight_decay']
num_epochs = study_st.best_trial.params['num_epochs']
study_st.best_trial.params

{'num_heads': 7,
 'model_dim': 392,
 'num_layers': 3,
 'dropout': 0.3360505706259301,
 'learning_rate': 0.003928416878757786,
 'weight_decay': 3.886347853547162e-05,
 'num_epochs': 19}

In [26]:
# single thread model initialization
model_st = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_st = model_st.to(DEVICE)
criterion_st = nn.MSELoss()
optimizer_st = optim.AdamW(model_st.parameters(), lr=lr, weight_decay=wd)

model_st.train()

for epoch in range(num_epochs):
	optimizer_st.zero_grad()
	output = model_st(X_st_train)
	loss = criterion_st(output, y_st_train)
	loss.backward()
	optimizer_st.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_st.eval()
		with torch.no_grad():
			val_predictions = model_st(X_st_test)
			val_loss = criterion_st(val_predictions, y_st_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_st.train()

Epoch 10/19, Loss: 17.42947769165039, Val Loss: 28.5018310546875
Epoch 19/19, Loss: 17.57338523864746, Val Loss: 40.58216857910156


## Multi Thread

In [27]:
# hyperparameters
num_heads = study_mm.best_trial.params['num_heads']
model_dim = study_mm.best_trial.params['model_dim']
num_layers = study_mm.best_trial.params['num_layers']
dropout = study_mm.best_trial.params['dropout']
lr = study_mm.best_trial.params['learning_rate']
wd = study_mm.best_trial.params['weight_decay']
num_epochs = study_mm.best_trial.params['num_epochs']

study_mm.best_trial.params

{'num_heads': 8,
 'model_dim': 264,
 'num_layers': 4,
 'dropout': 0.35614664010047575,
 'learning_rate': 0.009950757747095477,
 'weight_decay': 0.0008579245645970467,
 'num_epochs': 11}

In [28]:
# multi thread model initialization
model_mm = TransformerModel(input_dim, model_dim, num_heads, num_layers, output_dim, dropout)
if DEVICE.type == 'cuda':
	model_mm = model_mm.to(DEVICE)
criterion_mm = nn.MSELoss()
optimizer_mm = optim.AdamW(model_mm.parameters(), lr=lr, weight_decay=wd)

model_mm.train()

for epoch in range(num_epochs):
	optimizer_mm.zero_grad()
	output = model_mm(X_mm_train)
	loss = criterion_mm(output, y_mm_train)
	loss.backward()
	optimizer_mm.step()
	# validation
	if (epoch+1) % 10 == 0 or epoch == num_epochs-1:
		model_mm.eval()
		with torch.no_grad():
			val_predictions = model_mm(X_mm_test)
			val_loss = criterion_mm(val_predictions, y_mm_test)
		print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}')
		model_mm.train()

Epoch 10/11, Loss: 38.065513610839844, Val Loss: 19.448427200317383
Epoch 11/11, Loss: 41.414222717285156, Val Loss: 18.498815536499023


# Conclusion
Queda trabajo que hacer en la red, además de conseguir más datos para un entrenamiento más robusto. Queda por ahora descartado el uso de solo un modelo para multi-threading y single-threading, ya que el modelo tiene más del triple de *loss*.